# DOA(Direction of Arrival)

This notebook  will illustrates how to to estimate DOA(Direction of Arrival) use the [ReSpeaker 4-Mic Array](https://www.seeedstudio.com/ReSpeaker-4-Mic-Array-for-Raspberry-Pi-p-2941.html).

This example notebook does the following.

* import python libraries
* select RPi switch and using MicroblazeLibrary
* initialize AC108 Voice Capture ADCs
* find the DOA

It uses the [ReSpeaker 4-Mic Array](https://www.seeedstudio.com/ReSpeaker-4-Mic-Array-for-Raspberry-Pi-p-2941.html) to capture audio and to find the direction where the sound source is located.
![PYNQ-Z2 and ReSpeaker](./data/respeaker_pynq.jpg)

The overlay includes a custom IP core to transfer audio data.
![Block Design](./data/block_design.png)

 ### 1. ReSpeaker 4-Mic Array Introduction
 ReSpeaker 4-Mic Array is a 4 microphone expansion board designed for AI and voice applications. This means that you can build a more powerful and flexible voice product that integrates Amazon Alexa Voice Service, Google Assistant, and so on.

There are several algorithms such as DOA, VAD, NS and KWS we can use with the 4 mic array.
![PYNQ Z2 and ReSpeaker](./data/4_mic_array.jpg)

### 2. Prepare the overlay
Download the overlay first, then select the shared pin to be connected to RPi header (by default, the pins will be connected to PMODA instead).

In [1]:
from pynq import PL
from pynq import Overlay
from pynq import MMIO
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import wave
from respeaker import *

### 3. Initialize hardware
Load overlay and initialize the ReSpeaker by I2C.

The block design includes a ReSpeaker IP core to transfer the PCM/TDM format audio data. 

In [2]:
ol = Overlay("./bitstream/respeaker_wifi.bit")
ol.download()
ac108_init()

### 4. Define parameters
Define the custom IP address and audio data container

In [3]:
RESPEAKER_ADDR = 0x43C00000
RESPEAKER_RANGE = 0x200
RESPEAKER_OFFSET = 0x00
mmio = MMIO(RESPEAKER_ADDR, RESPEAKER_RANGE)
cap_cnt = 5000
ch1 = np.zeros(shape=(cap_cnt),dtype=np.uint32)
ch2 = np.zeros(shape=(cap_cnt),dtype=np.uint32)
ch3 = np.zeros(shape=(cap_cnt),dtype=np.uint32)
ch4 = np.zeros(shape=(cap_cnt),dtype=np.uint32)
ch5 = np.zeros(shape=(cap_cnt),dtype=np.uint32)

### 5. Find the DOA
capture the audio using mmio class and print the DOA

In [4]:
for t in range(0,30):
    for i in range(0,cap_cnt):
        ch1[i] = mmio.read(0)
        ch2[i] = mmio.read(4)
        ch3[i] = mmio.read(8)
        ch4[i] = mmio.read(12)
    ch1_i = np.int32(ch1[:])
    ch2_i = np.int32(ch2[:])
    ch3_i = np.int32(ch3[:])
    ch4_i = np.int32(ch4[:])
    close1 = np.allclose(ch1_i, ch3_i, 2, 8e7)
    close2 = np.allclose(ch2_i, ch4_i, 2, 8e7)
    if(close1 and close2):
        ;
    else:
        best_guess = get_direction(ch1_i, ch2_i, ch3_i, ch4_i)
        print("best guess of DOA is " + format(best_guess,'.2f') +  "°")

best guess of DOA is 178.14°
best guess of DOA is 135.30°
best guess of DOA is 45.30°
best guess of DOA is 358.14°
best guess of DOA is 284.70°
best guess of DOA is 61.86°
best guess of DOA is 315.30°
best guess of DOA is 262.35°
best guess of DOA is 178.14°
best guess of DOA is 178.14°
best guess of DOA is 14.70°
best guess of DOA is 210.00°
best guess of DOA is 120.00°
best guess of DOA is 135.30°
best guess of DOA is 194.70°
best guess of DOA is 151.86°
best guess of DOA is 178.14°
